In [6]:
#from pyscf import gto, scf, fci, lo
import netket as nk; import netket.experimental as nkx
import numpy as np
import itertools
import qiskit
from qiskit.quantum_info import Pauli, SparsePauliOp
from collections import defaultdict
import tensorflow as tf
import tensorflow as tf
import sys
import os
import pandas as pd
import itertools
from itertools import product
import matplotlib.pyplot as plt
#sys.path.append(os.path.dirname(os.getcwd()) + "/SAMPLER_LOCAL_IMPORT")
#from Sampling_Quantum import *
#from New_MCMC_Proposal import *
#sys.path.append(os.getcwd() + "/Code_download_Bell_2")
from MCMC_funs_Leyden import *

print(qiskit.version.get_version_info())
os.getcwd()

1.0.2


"c:\\Users\\vinit\\Downloads\\Research\\Quantum-Sampling\\Manas' Proposals"

In [7]:
def all_configs(n):
    return [np.array(v) for v in product([-1, 1], repeat=n)]



def Sampling_MCMC_trajectories(problem_inst, Transition_matrix, sample_size=10000, 
                               burn=1000, method='Quantum', init_config=None):

    n = problem_inst.n
    beta = problem_inst.T
    prob_dist = np.zeros(2**n)

    #exact_dist = np.exp(-beta * Proposal_object.Energy_array)
    #exact_dist = exact_dist / np.sum(exact_dist)

    #err_hist = []
    key_list = []

    if init_config==None:
        s = np.random.choice([1,-1],size=n)
    else: s = init_config

    int_key = spinconf2int(s)
    #print(int_key)
    
    for k in range(burn):
        #s = Proposal_object.generate_MCMC_trajectories(Transition_matrix, s)
         int_key = generate_move(transition_mat=Transition_matrix, state=int_key)

    for k in range(sample_size):
        #s = Proposal_object.generate_MCMC_trajectories(Transition_matrix, s)
        int_key = generate_move(transition_mat=Transition_matrix, state=int_key)
        #key = spinconf2int(s)
        prob_dist[int_key] +=1
        key_list.append(int_key)

    
    return np.flip(prob_dist/np.sum(prob_dist)), key_list

In [8]:
def grad_U_mala(x, J, h, alpha):
    v = np.tanh(alpha * x)
    sech_sq = 1.0 * (1 - v ** 2)
    return alpha * sech_sq * (h + 2 * J @ v)

def normalize_transition_matrix(T, eps=1e-12, verbose=False):
    T = T.copy()
    T[T < 0] = 0.0
    col_sums = T.sum(axis=0)
    bad = np.abs(col_sums) <= eps
    good = ~bad
    if np.any(good):
        T[:, good] /= col_sums[None, good]
    if np.any(bad):
        T[:, bad] = 1.0 / T.shape[0]
        if verbose:
            print(f"{bad.sum()} columns had near-zero sum. Reset to uniform.")
    return T


def Cov_computer(sample_vector_stacks):
    #sample_vector_stacks is a row-wise stack of last k sample vectors. Each sample vector of length n
    X = np.asarray(sample_vector_stacks, dtype=float)
    #print(X)
    if X.shape[0] <= 1:
        return np.eye(X.shape[1])
    mean = X.mean(axis=0)
    #print(np.tile(mean, (X.shape[0],1)))
    diffs = X - np.tile(mean, (X.shape[0],1))
    return (diffs.T @ diffs) / (X.shape[0] - 1)


def normal_dist_logpdf(y, mean, P, epsilon):
    diff = y - mean
    z = np.linalg.solve(P, diff)          # P z = diff  z = P^-1 diff
    quad_form = float(z @ z)
    logdet = 2.0*np.sum(np.log(np.diag(P))) + 2.0*y.size*np.log(epsilon)
    return -0.5 * (y.size * np.log(2.0 * np.pi) + logdet + (epsilon**(-2))*quad_form)


In [21]:
def adaptive_MALA_step_new(x, cov, epsilon, problem_inst, alpha=3.0):
    J_Q = problem_inst.J_quantum
    h_Q = problem_inst.h_quantum
    cov = cov + 1e-5 * np.eye(len(x))
    P = np.linalg.cholesky(cov)

    mean_shift_at_xnew = x + (0.5*epsilon**2) * problem_inst.T * cov @ grad_U_mala(x, J_Q, h_Q, alpha)

    #white_noise_term = np.random.multivariate_normal(np.zeros(len(x)), epsilon**2 * np.dot(P, np.conjugate(P).T))
    x_new = np.random.multivariate_normal(mean=mean_shift_at_xnew, cov=epsilon**2 * np.dot(P, np.conjugate(P).T), size=1)[0]
    #print(x_new)

    mean_shift_at_xold = x_new + (0.5*epsilon**2) * problem_inst.T * cov @ grad_U_mala(x_new, J_Q, h_Q, alpha)

    v_old = np.sign(np.tanh(alpha * x))
    v_new = np.sign(np.tanh(alpha * x_new))
    E_old = problem_inst.E_arr[::-1][spinconf2int(v_old)]
    E_new = problem_inst.E_arr[::-1][spinconf2int(v_new)]

    log_prop_dist_fwd = normal_dist_logpdf(x_new, mean_shift_at_xnew, P, epsilon)
    log_prop_dist_rev = normal_dist_logpdf(x, mean_shift_at_xold, P, epsilon)

    log_accept_ratio = -problem_inst.T * (E_new - E_old) + (log_prop_dist_rev - log_prop_dist_fwd)
    accept_prob = min(1.0, np.exp(log_accept_ratio))

    if np.random.rand() < accept_prob:
        return x_new, v_new
    else:
        return x, v_old



def adaptive_MALA_T_matrix(problem_inst, alpha=3.0, epsilon=0.3, num_samples=500):
    from tqdm import tqdm

    n = problem_inst.n
    #beta = problem_inst.T

    configs = all_configs(n)
    idx_map = {tuple(c): i for i, c in enumerate(configs)}
    T = np.zeros((1 << n, 1 << n))

    burn_in = 1000

    for v in tqdm(configs, desc="Building T matrix"):
        x = np.arctanh(np.clip(v, -0.999, 0.999)) / alpha
        cov = np.eye(n)

        counts = defaultdict(int)

        v_samples = []
        x_samples = []
        adapt_window = 20

        for step in range(1, num_samples+1):
            #x, v_new = adaptive_MALA_step(x, cov, epsilon, problem_inst, alpha=3.0)
            x, v_new = adaptive_MALA_step_new(x, cov, epsilon, problem_inst, alpha=3.0)
            v_samples.append(v_new)
            x_samples.append(x)

            key = tuple(v_new.astype(int))
            counts[key] = counts.get(key, 0) + 1

            if (step) % adapt_window == 0 and step > 200 and step <= burn_in:
                if len(x_samples) >= 80:
                    #x_hist = np.array([np.arctanh(np.clip(np.tanh(alpha * x0), -0.999, 0.999)) / alpha for x0 in v_samples[-50:]])
                    x_hist = [x for x in x_samples[-80:]]

                else:
                    #x_hist = np.array([np.arctanh(np.clip(np.tanh(alpha * x0), -0.999, 0.999)) / alpha for x0 in v_samples])
                    x_hist = [x for x in x_samples[:]]

                #cov_new = np.cov(np.array(x_hist).T) 
                #print(x_hist)

                cov_new = Cov_computer(x_hist)
                gamma = 0.1             # diminishing
                cov = (1.0 - gamma) * cov_new + gamma * cov
                cov = 0.5 * (cov + cov.T)

        i = idx_map[tuple(v)]
        total = sum(counts.values())

        for v_prime, c in counts.items():
            j = idx_map[v_prime]

            if j != i :
                T[j, i] = c / total

        T[i,i] = 1-sum(T[:,i])

    T = normalize_transition_matrix(T, eps=1e-12, verbose=True)

    return T



In [ ]:

from tqdm import tqdm
instances = pd.read_pickle('instances_new.pkl')
tot_instances = np.max(instances['Instance Number'].values)
print(tot_instances)
tot_instances = 10
results = pd.DataFrame(columns = [
    'Instance Number',
    'Spins', 
    'Temperature', 
    'Connectivity',
    'Proposal',
    'Acceptance',
    'Gap',
    'Gap Lazy'
        ])
T_logmesh = np.logspace(-3, 3, 50)
T_round = np.sort( np.kron(np.array([1,5]), 10.**np.arange(-3,4)))
T_lim = np.array([0, np.inf]) 
T_arr = np.unique( np.concatenate((T_logmesh, T_round, T_lim))) 
#T_arr = np.delete(T_arr, -2) 
T_arr = [1]
delta_step = 0.2

for instance_num in range(1, tot_instances+1):

    cond  = (instances['Instance Number']==instance_num)
    n            = instances[cond]['Spins'].values[0]
    connectivity = instances[cond]['Connectivity'].values[0]
    J            = instances[cond]['J'].values[0]
    h            = instances[cond]['h'].values[0]

    print('Starting problem instance', instance_num, 'of', tot_instances, 'with n =', n)

    problem_inst = ProblemInstance(J, h)
    problem_inst.T = 1.0

    proposal_mats = {}

    T = 1.0
    prop_type = "MALA_MC"
    accept_type = 'metropolis'


    transition_mat = adaptive_MALA_T_matrix(problem_inst, alpha=3.0, epsilon=0.2, num_samples=1000)

    gap, _ = abs_spectral_gap(transition_mat)

    results_datum = {
        'Instance Number': instance_num,
        'Spins': n, 
        'Temperature': T, 
        'Connectivity': connectivity,
        'Proposal': prop_type,
        'Acceptance': accept_type,
        'Gap': gap,
        "delta_time_step_Trotter": delta_step
    }


    results = pd.concat([results,pd.DataFrame([results_datum])], ignore_index=True)

    print("Instance {}, T = {}, Proposal = {}, Acceptance = {}, Gap = {:.4f}".format(
        instance_num, T, prop_type, accept_type, gap))

    #results = pd.concat([results,pd.DataFrame([results_datum])], ignore_index=True)

    results_row = pd.DataFrame([results_datum])
    results_row.to_csv('results_MALA.csv', mode='a', header=not os.path.exists('results_MALA.csv'), index=False)
    


2000
Starting problem instance 1 of 10 with n = 3


Building T matrix: 100%|██████████| 8/8 [00:01<00:00,  5.69it/s]
C:\Users\vinit\AppData\Local\Temp\ipykernel_43360\3949153596.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results,pd.DataFrame([results_datum])], ignore_index=True)


Instance 1, T = 1.0, Proposal = MALA_MC, Acceptance = metropolis, Gap = 0.0000
Starting problem instance 2 of 10 with n = 4


Building T matrix: 100%|██████████| 16/16 [00:02<00:00,  6.68it/s]


Instance 2, T = 1.0, Proposal = MALA_MC, Acceptance = metropolis, Gap = 0.0000
Starting problem instance 3 of 10 with n = 5


...

Building T matrix: 100%|██████████| 256/256 [00:43<00:00,  5.83it/s]


Instance 6, T = 1.0, Proposal = MALA_MC, Acceptance = metropolis, Gap = 0.0000
Starting problem instance 7 of 10 with n = 9


Building T matrix: 100%|██████████| 512/512 [01:30<00:00,  5.64it/s]


Instance 7, T = 1.0, Proposal = MALA_MC, Acceptance = metropolis, Gap = 0.0000
Starting problem instance 8 of 10 with n = 10


Building T matrix:  32%|███▏      | 330/1024 [04:54<05:48,  1.99it/s]